**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo
skvideo.setFFmpegPath('/Users/dobau/Desktop/MVA/S1/Deep Learning/ffmpeg/bin')
import skvideo.io
import cv2
import json
import collections

from tqdm import tqdm_notebook as tqdm

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense, Flatten
from keras.optimizers import sgd, Adam
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)
            
        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function  ```act``` selects the next action by mixing between the greedy policy, which selects the action that led to the best rewards so far, and the random policy which draws the action uniformly at random. This policy is called the $\epsilon$-greedy policy and is a widely studied policy which is the simplest policy that allows to handle the Exploration/Exploitation tradeoff in Reinforcement Learning.

The Exploitation consists in using our current knowledge to maximize the reward based on the past experience. On the other hand, the Exploration consists in trying new states in order to improve our comprehension of the environment. Both are crucial: the Exploitation is essential as the final goal is to maximize the reward, and the Exploration too as our past experience may not provide us sufficient information to properly understand the environment structure.

As I said, this algorithm may be the simplest policy to deal with this tradeoff, by just separating the two by chosing at each time to explore or exploit. The fact that it never stops to explore is bad for the reward in the long run as a random action causes a linear regret in average.

The mathematical formulation of this policy is:

\begin{equation}
a_{t+1} =  \left\{
    \begin{array}{cc}
          \text{arg}\max\limits_{a \in \mathcal{A}} Q(s_t, a)  & \text{with probability } 1-\epsilon\\  
          U[\mathcal{A}] & \text{with probability } \epsilon\\ 
    \end{array}
\right.
\end{equation}

The parameter  ```epsilon``` is crucial as it tunes the level of Exploration/Exploitation. Its optimal level depends on the problem structure.


***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [6]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self, e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self, t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:]  = 0
        b[:,:2,:]  = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 # error (fixed)
        self.position[:, -2:] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [7]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=100 # set small when debugging
epochs_test=100 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The purpose of these two arrays is to propose a space representation of the cells of the board, the position of the rat and the position of the different items using 2D matrices. They are defined as follow:

* `position` is an array that contains 0 in every cell that has not been visited by the rat and 1 otherwise. It is storing the path of the rat, and contains -1 at the borders to indicate the limits of the map

* `board` is an array that represents the position of all of the rewards. It contains 0 in every empty cell, 1 for the position of a bonus and -0.5 for a malus. If the rat visits a cell then its content is set at 0.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [8]:
class RandomAgent(Agent):
    def __init__(self, n_action=4):
        super(RandomAgent, self).__init__(n_action)
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action)
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [9]:
def test(agent, env, epochs, prefix='', frame=10):
    # Number of won games
    score = 0
        
    for e in range(1, epochs+1):
        state = env.reset()
        game_over = False
        win, lose = 0, 0
        
        while not game_over:
            action = agent.learned_act(state)
            prev_state = state
            state, reward, game_over = env.act(action)
            
            if reward > 0:
                win += reward
            elif reward < 0:
                lose -= reward
                
        # Update stats
        score += win-lose
        
        # Save as a mp4
        if e%frame == 0:
            env.draw(prefix+str(e))
            print("Epoch {:03d}/{:03d} | Win/lose count {}/{} ({})"
              .format(e, epochs, win, lose, win-lose))

    print('Final score: '+str(score/epochs))
    return score/epochs

In [10]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random70.mp4'))

Epoch 010/100 | Win/lose count 10.0/20.0 (-10.0)
Epoch 020/100 | Win/lose count 9.0/11.0 (-2.0)
Epoch 030/100 | Win/lose count 12.0/23.0 (-11.0)
Epoch 040/100 | Win/lose count 9.5/19.0 (-9.5)
Epoch 050/100 | Win/lose count 8.0/20.0 (-12.0)
Epoch 060/100 | Win/lose count 6.5/19.0 (-12.5)
Epoch 070/100 | Win/lose count 7.0/11.0 (-4.0)
Epoch 080/100 | Win/lose count 9.0/11.0 (-2.0)
Epoch 090/100 | Win/lose count 12.0/9.0 (3.0)
Epoch 100/100 | Win/lose count 6.0/14.0 (-8.0)
Final score: -3.59


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




a)
First, we easily get by isolating the first term:
\begin{equation}
\begin{array}{lcl}
    Q^{\pi}(s, a) &=& \mathbb{E}_{P^{\pi}} \Bigg[ \displaystyle \sum_{t\geq0}^{\infty} \gamma^t r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg] \\
    & = & r(s,a) + \gamma \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\
\end{array}
\end{equation}
Then, by isolating the right term:
\begin{equation}
\begin{array}{lcl}
   && \mathbb{E}_{P^{\pi}} \Bigg[\displaystyle \sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a \Bigg]\\ &=& \displaystyle \sum_{(a',s')}  \mathbb{E}_{P^{\pi}} \Bigg[\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_0=s, a_0=a, s_1=s', a_1=a' \Bigg] \times \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big)\\
    &=& \displaystyle \sum_{(a',s')}  \mathbb{E}_{P^{\pi}} \Bigg[\sum_{t\geq1}^{\infty} \gamma^{t-1} r(s_t, a_t) \Big| s_1=s', a_1=a' \Bigg] \times \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big) \\
    &=& \displaystyle \sum_{(a',s')} \mathbb{P}\Big(s_1=s', a_1=a'\Big|s_0=s, a_0=a\Big) \times  Q^{\pi}(s', a') \\
    &=& \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} \big[ Q^{\pi}(s', a') \big] \\
\end{array}
\end{equation}

We finally get:
$$Q^{\pi}(s, a) = \mathbb{E}_{(s',a')\sim P(\cdot|s,a)} \big[ r(s,a) + \gamma Q^{\pi}(s', a') \big]$$

____

b)

Proving this equality comes directly from proving the following one:
\begin{equation*} \max_{\pi}\mathbb{E}_{(s',a')\sim p(\cdot|s,a)}[Q^{\pi}(s',a')]=\mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}[\max_{a'}Q^*(s',a')] \\
= \mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}[\max_{a'}\max_{\pi}Q^{\pi}(s',a')]
\end{equation*}

First of all, the Jensen inequality, proves that

\begin{equation*} \max_{\pi}\mathbb{E}_{(s',a')\sim p(\cdot|s,a)}[Q^{\pi}(s',a')] \leq \mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}[\max_{a'}Q^*(s',a')] \end{equation*}

The following inequality achieves to prove the result:

\begin{align*}\mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[\max_{a''}Q^*(s',a'')] &= \max_{a''}\mathbb{E}_{s'' \sim \pi^*(\cdot|s,a)}[Q^{\pi^*}(s',a'')] \\
&\leq \max_{a''}\max_{\pi}\mathbb{E}_{(s',a')\sim p^{\pi}(\cdot|s,a)}[Q^{\pi}(s',a')] \\
&\leq \max_{\pi}\mathbb{E}_{(s',a')\sim p^{\pi}(\cdot|s,a)}[Q^{\pi}(s',a')] 
\end{align*}
___

c)

Let's denote: $Q(s, a; \theta) \approx Q^{*}(s,a)$ where $\theta$ is the set of hyperparameters.
The minimization of the average squarred-error between $y^{(t)}$ and $Q(s, a; \theta^{(t)})$ gives:
\begin{align*}
\mathcal{L}_t(\theta^{(t)}) = \mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}[(y^{(t)} - Q(s, a; \theta^{(t)}))^2]
\end{align*}

Then $y^{(t)}$ is chosen as the $\mathcal{Q}$-value of the best action that would have been chosen at the next iteration using $\theta^{(t-1)}$:

\begin{align*}
y^{(t)} = \mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}[r(s,a)+\gamma \max_{a'} Q(s',a'; \theta^{(t-1)})]
\end{align*}

Finally we get:
\begin{align*} \mathcal{L}(\theta^{(t)})=\mathbb{E}_{s' \sim \pi^*(\cdot|s,a)}\Bigg[\Big( r(s,a)+\gamma\max{a'}Q(s',a',\theta^{(t-1)})-Q(s,a,\theta^{(t)})\Big)^{2}\Bigg]
\end{align*}

Hence the result.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [11]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = collections.deque(maxlen=max_memory) 
        
    def remember(self, m):
        self.memory.append(m)

    def random_access(self):
        idx = np.random.randint(len(self.memory))
        return self.memory[idx]

***
The pipeline we will use for training is given below:

In [12]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Update stats
        score += win-lose
        
        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))
        
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
            
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [13]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)
        # Discount for Q learning
        self.discount = 0.99        
        self.grid_size = grid_size        
        # number of state
        self.n_state = n_state
        # Memory
        self.memory = Memory(memory_size)      
        # Batch size when learning
        self.batch_size = batch_size

    # add epsilon decay method for last question
    def set_epsilon(self, e):
        self.epsilon *= e # e is the decay percentage
        
    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, 0)))

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool
        self.memory.remember([s_, n_s_, a_, r_, game_over_])       
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            # sample minibatch of transitions
            # (state, next_state, action, reward, game_over)
            s_, n_s, a_, r_, game_over_ = self.memory.random_access()
            input_states[i] = s_
            target_q[i] = self.model.predict(np.expand_dims(s_, 0))            
            if game_over_:
                target_q[i, a_] = r_
            else:
                target_q[i, a_] = r_ + self.discount * np.max(self.model.predict(np.expand_dims(n_s, 0)))
        target_q = np.clip(target_q, -3, 3)
        l = self.model.train_on_batch(input_states, target_q)


        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("adam", "mse")
        self.model = model

            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1, **kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        model = Sequential()
        model.add(Dense(20, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Flatten())
        model.add(Dense(10, activation='relu'))
        model.add(Dense(4))
        adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(adam, "mse")
        self.model = model

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(grid_size=size, lr=.001, epsilon = 0.1, memory_size=50000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train90.mp4'))

Epoch 000/100 | Loss 0.0558 | Win/lose count 4.5/4.0 (0.5)
Epoch 001/100 | Loss 0.0077 | Win/lose count 3.5/8.0 (-4.5)
Epoch 002/100 | Loss 0.0065 | Win/lose count 8.5/1.0 (7.5)
Epoch 003/100 | Loss 0.0143 | Win/lose count 4.5/11.0 (-6.5)
Epoch 004/100 | Loss 0.0028 | Win/lose count 4.0/5.0 (-1.0)
Epoch 005/100 | Loss 0.0019 | Win/lose count 5.0/4.0 (1.0)
Epoch 006/100 | Loss 0.0088 | Win/lose count 6.0/5.0 (1.0)
Epoch 007/100 | Loss 0.0047 | Win/lose count 2.0/1.0 (1.0)
Epoch 008/100 | Loss 0.0031 | Win/lose count 8.5/4.0 (4.5)
Epoch 009/100 | Loss 0.0433 | Win/lose count 7.5/3.0 (4.5)
Epoch 010/100 | Loss 0.0481 | Win/lose count 5.5/3.0 (2.5)
Epoch 011/100 | Loss 0.0028 | Win/lose count 3.0/0 (3.0)
Epoch 012/100 | Loss 0.0077 | Win/lose count 6.5/2.0 (4.5)
Epoch 013/100 | Loss 0.0036 | Win/lose count 3.0/4.0 (-1.0)
Epoch 014/100 | Loss 0.0041 | Win/lose count 7.5/2.0 (5.5)
Epoch 015/100 | Loss 0.0023 | Win/lose count 8.0/1.0 (7.0)
Epoch 016/100 | Loss 0.0035 | Win/lose count 4.0/3.0 

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [15]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        model = Sequential()
        model.add(Conv2D(32, 3, input_shape=(5, 5, self.n_state), activation='relu'))
        model.add(Conv2D(64, 3, activation='relu'))
        model.add(Flatten())
        model.add(Dense(50, activation='relu'))
        model.add(Dense(4))
        adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
        model.compile(adam, "mse")
        self.model = model

In [16]:
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=50000, batch_size = 32)
train(agent, env, epochs_train, prefix='cnn_train')
HTML(display_videos('cnn_train90.mp4'))

Epoch 000/100 | Loss 0.0013 | Win/lose count 3.5/7.0 (-3.5)
Epoch 001/100 | Loss 0.0033 | Win/lose count 6.0/6.0 (0.0)
Epoch 002/100 | Loss 0.0013 | Win/lose count 7.0/4.0 (3.0)
Epoch 003/100 | Loss 0.0025 | Win/lose count 9.0/4.0 (5.0)
Epoch 004/100 | Loss 0.0020 | Win/lose count 12.5/6.0 (6.5)
Epoch 005/100 | Loss 0.0072 | Win/lose count 15.0/2.0 (13.0)
Epoch 006/100 | Loss 0.0040 | Win/lose count 16.0/5.0 (11.0)
Epoch 007/100 | Loss 0.0047 | Win/lose count 17.0/2.0 (15.0)
Epoch 008/100 | Loss 0.0040 | Win/lose count 12.5/3.0 (9.5)
Epoch 009/100 | Loss 0.0021 | Win/lose count 13.5/3.0 (10.5)
Epoch 010/100 | Loss 0.0040 | Win/lose count 2.0/0 (2.0)
Epoch 011/100 | Loss 0.0082 | Win/lose count 17.0/0 (17.0)
Epoch 012/100 | Loss 0.0016 | Win/lose count 10.0/0 (10.0)
Epoch 013/100 | Loss 0.0014 | Win/lose count 9.0/4.0 (5.0)
Epoch 014/100 | Loss 0.0015 | Win/lose count 19.0/7.0 (12.0)
Epoch 015/100 | Loss 0.0041 | Win/lose count 12.0/2.0 (10.0)
Epoch 016/100 | Loss 0.0027 | Win/lose coun

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=50000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5', name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=50000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5', name_model='fc_trainmodel.json')
print('Test of the CNN')
score_cnn = test(agent_cnn, env, epochs_test, prefix='cnn_test')
print('Test of the FC')
score_fc = test(agent_fc, env, epochs_test, prefix='fc_test')

Test of the CNN
Epoch 010/100 | Win/lose count 7.5/0 (7.5)
Epoch 020/100 | Win/lose count 3.5/0 (3.5)
Epoch 030/100 | Win/lose count 5.5/0 (5.5)
Epoch 040/100 | Win/lose count 1.5/0 (1.5)
Epoch 050/100 | Win/lose count 7.0/0 (7.0)
Epoch 060/100 | Win/lose count 1.5/0 (1.5)
Epoch 070/100 | Win/lose count 2.0/0 (2.0)
Epoch 080/100 | Win/lose count 10.5/0 (10.5)
Epoch 090/100 | Win/lose count 1.5/0 (1.5)
Epoch 100/100 | Win/lose count 6.5/0 (6.5)
Final score: 4.345
Test of the FC
Epoch 010/100 | Win/lose count 3.0/0 (3.0)
Epoch 020/100 | Win/lose count 9.5/0 (9.5)
Epoch 030/100 | Win/lose count 4.5/0 (4.5)
Epoch 040/100 | Win/lose count 3.0/0 (3.0)
Epoch 050/100 | Win/lose count 5.0/0 (5.0)
Epoch 060/100 | Win/lose count 2.5/0 (2.5)
Epoch 070/100 | Win/lose count 4.0/0 (4.0)
Epoch 080/100 | Win/lose count 1.0/0 (1.0)
Epoch 090/100 | Win/lose count 8.0/0 (8.0)
Epoch 100/100 | Win/lose count 3.5/0 (3.5)
Final score: 3.32


In [18]:
HTML(display_videos('cnn_test90.mp4'))

In [19]:
HTML(display_videos('fc_test90.mp4'))

In [20]:
def temp_impact(epochs_test, n_expe):
    score_fc, score_cnn = np.zeros((10, 2)), np.zeros((10, 2))
    for i, temp in tqdm(enumerate(0.1*np.arange(1, 11)), total=10):
        env = Environment(grid_size=size, max_time=T, temperature=temp)
        s_fc, s_cnn = [], []
        for xp in range(n_expe):
            s_fc.append(test(agent_fc, env, epochs_test, prefix='fc_test', frame=1e3))
            s_cnn.append(test(agent_cnn, env, epochs_test, prefix='fc_test', frame=1e3))
        score_fc[i, 0]  = np.mean(s_fc)
        score_cnn[i, 0] = np.mean(s_cnn)
        score_fc[i, 1]  = np.std(s_fc)
        score_cnn[i, 1] = np.std(s_cnn)
    return score_fc, score_cnn

score_fc, score_cnn = temp_impact(75, 25)

Final score: 1.4066666666666667
Final score: 1.4066666666666667
Final score: 1.42
Final score: 1.1733333333333333
Final score: 1.4066666666666667
Final score: 1.4066666666666667
Final score: 1.4466666666666668
Final score: 1.2066666666666668
Final score: 1.3133333333333332
Final score: 1.3533333333333333
Final score: 1.4133333333333333
Final score: 1.5266666666666666
Final score: 1.4066666666666667
Final score: 1.2666666666666666
Final score: 1.54
Final score: 1.5666666666666667
Final score: 1.2466666666666666
Final score: 1.3733333333333333
Final score: 1.52
Final score: 1.48
Final score: 1.3466666666666667
Final score: 1.0533333333333332
Final score: 1.44
Final score: 1.4733333333333334
Final score: 1.4
Final score: 1.46
Final score: 1.3333333333333333
Final score: 1.3266666666666667
Final score: 1.58
Final score: 1.34
Final score: 1.5066666666666666
Final score: 1.3133333333333332
Final score: 1.2533333333333334
Final score: 1.5333333333333334
Final score: 1.4466666666666668
Final s

Final score: 10.026666666666667
Final score: 13.513333333333334
Final score: 8.953333333333333
Final score: 15.773333333333333
Final score: 8.646666666666667
Final score: 11.833333333333334
Final score: 8.746666666666666
Final score: 14.386666666666667
Final score: 9.72
Final score: 14.22
Final score: 8.9
Final score: 12.373333333333333
Final score: 9.56
Final score: 11.94
Final score: 9.893333333333333
Final score: 11.653333333333334
Final score: 8.826666666666666
Final score: 13.0
Final score: 8.626666666666667
Final score: 14.253333333333334
Final score: 9.446666666666667
Final score: 13.32
Final score: 11.193333333333333
Final score: 12.073333333333334
Final score: 10.86
Final score: 13.06
Final score: 9.78
Final score: 14.066666666666666
Final score: 9.213333333333333
Final score: 15.686666666666667
Final score: 10.106666666666667
Final score: 12.873333333333333
Final score: 8.326666666666666
Final score: 12.653333333333334
Final score: 9.98
Final score: 14.38
Final score: 9.66666

The CNN seems to achieve better results in comparable environments. But the improvement is actually not very significant. Both methods lead to clearly identified caveats, such as the fact that he can stay stuck in a place where there is no reward as he does not remember his path and wants to avoid negative rewards. Actually, the exploration part here is very poor and we could think of a way of implementing a strategy that would explore the space more efficiently.

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



```python
## use those samples of code:
#In train explore:
state, reward, game_over = env.act(action, train=True)

## In Environment exploring:
# You will have to change n_state to 3 because you will use one more layer!
reward = 0
if train:
    reward = -self.malus_position[self.x, self.y]
self.malus_position[self.x, self.y] = 0.1

reward = reward + self.board[self.x, self.y]
# 3 "feature" states instead of 2
state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                        self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

```

In [21]:
def train_explore(agent, env, epoch, epsilon=0.1, decay=0.99, prefix=''):
    score = 0
    for e in range(epoch):
        state = env.reset()
        game_over = False
        win, lose = 0, 0
        agent.set_epsilon(epsilon)
        while not game_over:
            action = agent.act(state)
            agent.set_epsilon(decay) 
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            if reward > 0:
                win += reward
            if reward < 0:
                lose -= reward
            loss = agent.reinforce(prev_state, state, action, reward, game_over)
        if e % 10 == 0:
            env.draw(prefix+str(e))
        score += win-lose
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {:3.2f}/{:3.2f} ({:3.2f})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the rat
        self.x = 0
        self.y = 1
        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0]    = 256
        b[self.board<0, 2] = 256
        b[self.x,self.y,:]   = 256
        b[-2:,:,:] = 0
        b[:,-2:,:] = 0
        b[:2,:,:]  = 0
        b[:,:2,:]  = 0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1 
        self.position[:, -2:] = -1
        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
        self.t = self.t + 1       
        reward = 0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] += 0.1
        reward = reward + self.board[self.x, self.y]       
        self.board[self.x, self.y] = 0 
        game_over = self.t > self.max_time 
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)     
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

    def reset(self):
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)
        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)
        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, 
                                 self.grid_size*self.scale, 3))
        malus[bonus>0]=0 
        self.board = bonus + malus 
        self.board[:2,:], self.board[-2:,:] = 0, 0
        self.board[:, :2], self.board[:, -2:]  = 0, 0
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:], self.position[:,0:2] = -1, -1
        self.position[-2:, :], self.position[:, -2:] = -1, -1
        self.board[self.x,self.y] = 0
        self.t = 0   
        self.malus_position = np.zeros((self.grid_size, self.grid_size))
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                                self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [22]:
# Training
size = 13
T = 200
epochs_train = 100
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon=0.1, memory_size=50000, batch_size=32, n_state=3)
train_explore(agent, env, epochs_train, decay=0.99, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore90.mp4'))

Epoch 000/100 | Loss 0.0380 | Win/lose count 7.00/67.50 (-60.50)
Epoch 001/100 | Loss 0.0811 | Win/lose count 18.00/39.80 (-21.80)
Epoch 002/100 | Loss 0.0108 | Win/lose count 21.00/28.50 (-7.50)
Epoch 003/100 | Loss 0.0083 | Win/lose count 20.00/26.30 (-6.30)
Epoch 004/100 | Loss 0.0079 | Win/lose count 25.50/25.20 (0.30)
Epoch 005/100 | Loss 0.0090 | Win/lose count 21.50/17.80 (3.70)
Epoch 006/100 | Loss 0.0216 | Win/lose count 20.00/25.80 (-5.80)
Epoch 007/100 | Loss 0.0235 | Win/lose count 23.00/14.90 (8.10)
Epoch 008/100 | Loss 0.0095 | Win/lose count 20.50/17.20 (3.30)
Epoch 009/100 | Loss 0.0053 | Win/lose count 21.50/12.60 (8.90)
Epoch 010/100 | Loss 0.0152 | Win/lose count 24.50/18.00 (6.50)
Epoch 011/100 | Loss 0.0137 | Win/lose count 30.00/9.90 (20.10)
Epoch 012/100 | Loss 0.0105 | Win/lose count 22.50/15.00 (7.50)
Epoch 013/100 | Loss 0.0137 | Win/lose count 23.50/20.00 (3.50)
Epoch 014/100 | Loss 0.0102 | Win/lose count 22.00/19.10 (2.90)
Epoch 015/100 | Loss 0.0113 | Win/

In [23]:
# Evaluation
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore90.mp4'))

Epoch 010/100 | Win/lose count 27.5/6.0 (21.5)
Epoch 020/100 | Win/lose count 25.0/4.0 (21.0)
Epoch 030/100 | Win/lose count 23.0/3.0 (20.0)
Epoch 040/100 | Win/lose count 19.5/2.0 (17.5)
Epoch 050/100 | Win/lose count 19.0/3.0 (16.0)
Epoch 060/100 | Win/lose count 18.5/7.0 (11.5)
Epoch 070/100 | Win/lose count 22.5/3.0 (19.5)
Epoch 080/100 | Win/lose count 30.0/2.0 (28.0)
Epoch 090/100 | Win/lose count 22.5/5.0 (17.5)
Epoch 100/100 | Win/lose count 24.0/2.0 (22.0)
Final score: 21.095


The results are actually much better with this new method. It seems to achieve the best possible performance.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.